## Load Census Data

In [1]:
import rpy2.robjects as robjects
import pandas as pd
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

robjects.r("source('lib/DataGathering.r')")

make_acs_table_t_r = robjects.globalenv["make_acs_table_t"]
make_acs_table_bg_r = robjects.globalenv["make_acs_table_bg"]

# Convert the R DataFrame to a pandas DataFrame
with localconverter(robjects.default_converter + pandas2ri.converter):
    make_acs_table_t = robjects.conversion.rpy2py(make_acs_table_t_r)
    make_acs_table_bg = robjects.conversion.rpy2py(make_acs_table_bg_r)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


R[write to console]: To install your API key for use in future sessions, run this function with `install = TRUE`.



In [2]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

In [3]:
acs_table_t_r = make_acs_table_t(2020)
acs_table_bg_r = make_acs_table_bg(2020)

R[write to console]: Getting data from the 2016-2020 5-year ACS

R[write to console]: Downloading feature geometry from the Census website.  To cache shapefiles for use in future sessions, set `options(tigris_use_cache = TRUE)`.

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'



  |======================================================================| 100%


R[write to console]: Getting data from the 2016-2020 5-year ACS

R[write to console]: Downloading feature geometry from the Census website.  To cache shapefiles for use in future sessions, set `options(tigris_use_cache = TRUE)`.

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'

R[write to console]: Using FIPS code '37' for state 'NC'

R[write to console]: Using FIPS code '063' for 'Durham County'



Downloading: 34 kB     34 kB     49 kB     49 kB     49 kB     49 kB     49 kB     49 kB     190 kB     190 kB     190 kB     190 kB     190 kB     190 kB     250 kB     250 kB     250 kB     250 kB     250 kB     250 kB     250 kB     250 kB     280 kB     280 kB     320 kB     320 kB     380 kB     380 kB     380 kB     380 kB     450 kB     450 kB     450 kB     450 kB     450 kB     450 kB     470 kB     470 kB     510 kB     510 kB     510 kB     510 kB     540 kB     540 kB     570 kB     570 kB     570 kB     570 kB     610 kB     610 kB     640 kB     640 kB     640 kB     640 kB     640 kB     640 kB     640 kB     640 kB     660 kB     660 kB     690 kB     690 kB     690 kB     690 kB     730 kB     730 kB     760 kB     760 kB     760 kB     760 kB     770 kB     770 kB     830 kB     830 kB     830 kB     830 kB     830 kB     830 kB     840 kB     840 kB     840 kB     840 kB     890 kB     890 kB     890 kB     890 kB     890 kB     890 kB     890 kB     890 kB     910 k

In [4]:
# Convert the R DataFrame to a pandas DataFrame
with localconverter(robjects.default_converter + pandas2ri.converter):
    acs_table_t = robjects.conversion.rpy2py(acs_table_t_r)
    acs_table_bg = robjects.conversion.rpy2py(acs_table_bg_r)

In [5]:
from lib.data import (
    get_parcels,
    get_du_est,
    add_columns_from_csv,
    add_columns_from_census,
)

In [6]:
durham_open = get_parcels()
parcels_clean = get_du_est()

/Users/dhavalpotdar/Workarea/Data+/DPS/durham-public-schools/src/lib/dataloader_alex.py:14: DtypeWarning: Columns (44,45,46,48,63,66,67,91,109,111,119) have mixed types. Specify dtype option on import or set low_memory=False.
  du_df = pd.read_csv(PATH_DU_EST)


## Joins

In [24]:
def add_columns_from_csv(gdf, csv):

    gdf["REID"] = gdf["REID"].astype(str)
    csv["REID"] = csv["REID"].astype(str)

    merged_csv_gdf = gdf.merge(
        csv[
            [
                "REID",
                "designation",
                "housing_type",
                "du_est_final",
                "students2324",
                "students2223",
                "students2122",
                "students2021",
                "geo_id_b2010",
                "geo_id_b2020",
                "geo_id_bg2010",
                "geo_id_bg2020",
                "sch_id_base1819_es",
                "sch_id_base_es",
                "sch_id_gt_es",
                "sch_id_yr_es",
                "sch_id_yr_optout_es",
                "sch_id_zone",
                "sch_id_base_hs",
                "sch_id_gt_hs",
                "sch_id_base1819_ms",
                "sch_id_base_ms",
                "sch_id_gt_ms",
                "sch_id_yr_ms",
                "pu_2122_833",
                "pu_2324_848",
                "geo_id_t2010",
                "geo_id_t2020",
                "TOTAL_PROP_VALUE",
            ]
        ],
        left_on="REID",
        right_on="REID",
        how="right",
    )

    merged_csv_gdf.dropna(subset="OBJECTID_1", inplace=True)

    return merged_csv_gdf

In [25]:
base_dataset = add_columns_from_csv(durham_open, parcels_clean)

In [26]:
base_dataset

,OBJECTID_1,OBJECTID,FEATURE_KE,PARCEL_TYP,REID,PARCEL_PK,AUT_SNAPSH,PIN,PIN_EXT,PIN_MAP,PIN_BLOCK,PIN_LOT,PROPERTY_D,LOCATION_A,OLD_MAP,VCS,CITY,CITY_PCNT,FIRE_DISTR,FIRE_PCNT,SPEC_DIST,SPEC_DIST_,TOWNSHIP,LAND_CLASS,ACREAGE,CALCULATED,DEEDED_ACR,PERMIT_DAT,PERMIT_NUM,ZONING,HISTORY_RE,HISTORY__1,ETJ,PROPERTY_O,OWNER_MAIL,OWNER_MA_1,OWNER_MA_2,OWNER_MA_3,OWNER_MA_4,OWNER_MA_5,OWNER_MA_6,PHYADDR_LO,PHYADDR_ST,PHYADDR_DI,PHYADDR__1,PHYADDR__2,PHYADDR__3,PHYADDR_CI,PHYADDR__4,PHYADDR_ZI,PHYADDR_PL,TOTAL_LAND,TOTAL_BLDG,LAND_USE_V,USE_VALUE_,HISTORIC_V,TOTAL_DEFE,TOTAL_PROP,COST_TOTAL,INCOME_TOT,SALES_COMP,VALUE_APPR,DEED_DATE,DEED_BOOK,DEED_PAGE,REVENUE_ST,PKG_SALE_D,PKG_SALE_P,LAND_SALE_,LAND_SAL_1,TOTAL_UNIT,RECYCLE_UN,APT_SC_SQR,HEATED_ARE,ELDERLY_EX,OTHER_EXMP,NEIGHBORHO,VETRANS_EX,DEED_PATH,ACCOUNT_TY,CITY_CODE,FIRE_DIS_1,PHYADDR__5,TOTAL_OBLD,EXEMPTION_,MAP_BOOK,MAP_PAGE,MAP_SCALE,MAP_ROD_DO,IS_PENDING,GROSS_LEAS,PLAT_BOOK,PLAT_PAGE,PHOTO_PATH,PHOTO_DATE,Shape__Are,Shape__Len,geometry,designation,housing_type,du_est_final,students2324,students2223,students2122,students2021,geo_id_b2010,geo_id_b2020,geo_id_bg2010,geo_id_bg2020,sch_id_base1819_es,sch_id_base_es,sch_id_gt_es,sch_id_yr_es,sch_id_yr_optout_es,sch_id_zone,sch_id_base_hs,sch_id_gt_hs,sch_id_base1819_ms,sch_id_base_ms,sch_id_gt_ms,sch_id_yr_ms,pu_2122_833,pu_2324_848,geo_id_t2010,geo_id_t2020,TOTAL_PROP_VALUE
0,5293332.0,78584.0,78587.0,0.0,100000,1.0,2021-12-20,0822419440,000,0822,41,9440,PROP-DUKE UNIV/EAST CAMPU S-N/S W MAIN ST DUKE...,1500 W MAIN ST,None,C822N,DURHAM,100,None,None,CAPITAL FINANCE,100,NONE,CMNTY SVC/ COLLEGE,97.24,97.24,97.24,None,None,UC(D),None,None,DURHAM CITY,DUKE UNIVERSITY,"2200 WEST MAIN ST, STE 300",None,None,DURHAM,NC,27705,None,3339.0,1500,W,MAIN,ST,None,DURHAM,NC,27705,None,29471700.0,0.0,29471700.0,NaN,0.0,0.0,29471700.0,29471700.0,None,None,VLAPCOST,1997-01-01,000013,00335,0.0,None,NaN,None,NaN,None,None,None,NaN,None,29471700.0,W PETTIGREW ST,None,000013/,None,None,None,None,0.0,0200,None,None,00,None,N,0.0,000000,000000,None,None,4.241395e+06,8814.872258,"MULTIPOLYGON (((-78.91250 36.00508, -78.91253 ...",non-res,non-res,0,4,6,2,1,3.706300e+14,3.706300e+14,3.706300e+11,3.706300e+11,363.0,363.0,363.0,364.0,NaN,347.0,365.0,365.0,306.0,306.0,306.0,370.0,16.0,15.0,3.706300e+10,3.706300e+10,29471700.0
1,5293333.0,18719.0,18719.0,0.0,100001,2.0,2021-12-20,0822537639,000,0822,53,7639,THE SUBURBAN L&P CO/BLK:1 5/LT#01 PL00005A-000041,922 LANCASTER ST,None,R822D,DURHAM,100,None,None,CAPITAL FINANCE,100,NONE,RES/ 2-FAMILY,0.16,0.16,0.16,None,None,RU-5(2),None,None,DURHAM CITY,"WILKERSON, GLORIA MCBROOM",2429 BIVINS RD,None,None,DURHAM,NC,27712,None,3069.0,922,None,LANCASTER,ST,None,DURHAM,NC,27701,None,115900.0,45120.0,115900.0,NaN,0.0,0.0,161020.0,161020.0,None,None,VLAPCOST,2008-07-09,006008,00920,0.0,None,NaN,None,NaN,None,None,None,1216.0,None,0.0,WALL TOWN,None,006008/,None,None,None,None,0.0,None,None,None,00,None,N,0.0,00005A,000041,None,None,6.934546e+03,376.739897,"POLYGON ((-78.91321 36.01309, -78.91320 36.013...",multi,du_tri,2,0,0,0,0,3.706300e+14,3.706300e+14,3.706300e+11,3.706300e+11,363.0,363.0,363.0,364.0,NaN,347.0,365.0,365.0,306.0,306.0,306.0,370.0,16.0,815.0,3.706300e+10,3.706300e+10,161020.0
2,5293334.0,18271.0,18271.0,0.0,100002,3.0,2021-12-20,0822537634,000,0822,53,7634,THE SUBURBAN L&P CO/BLK:1 5/LT#03 PL00005A-000041,920 LANCASTER ST,None,R822D,DURHAM,100,None,None,CAPITAL FINANCE,100,NONE,RES/ 2-FAMILY,0.16,0.16,0.16,None,None,RU-5(2),None,None,DURHAM CITY,SCANLON REALTY CORP,117 PARK AVENUE,None,None,SOUTHPORT,NC,28461,None,3069.0,920,None,LANCASTER,ST,None,DURHAM,NC,27701,None,115900.0,64657.0,115900.0,NaN,0.0,0.0,180557.0,180557.0,None,None,VLAPCOST,1997-01-01,001456,00111,0.0,None,NaN,None,NaN,None,None,None,1216.0,None,0.0,WALL TOWN,None,001456/,None,None,None,None,0.0,None,None,None,00,None,N,0.0,00005A,000041,None,None,6.917429e+03,376.639565,"POLYGON ((-78.91274 36.01307, -78.91275 36.012...",multi,du_tri,2

In [27]:
base_dataset["OBJECTID_1"].isna().sum()

0

In [28]:
def fix_geoid_dtypes(series):
    return series.fillna(-1).astype(int).astype(str)


base_dataset["geo_id_t2020"] = fix_geoid_dtypes(base_dataset["geo_id_t2020"])
base_dataset["geo_id_b2020"] = fix_geoid_dtypes(base_dataset["geo_id_b2020"])
base_dataset["geo_id_bg2020"] = fix_geoid_dtypes(base_dataset["geo_id_bg2020"])

In [29]:
def add_columns_from_census(gdf, csv, census_type="t"):

    census_col_list = [
        "GEOID",
        "estimate_rent_total",
        "moe_rent_total",
        "estimate_median_house_value",
        "estimate_median_year_structure_build",
        "estimate_housing_units",
        "pct_vacant",
        "pct_owner_occupied",
    ]

    csv = csv[census_col_list]

    if census_type == "t":

        csv = csv.add_suffix("_t")

        merged_csv_gdf = gdf.merge(
            csv,
            left_on="geo_id_t2020",
            right_on="GEOID_t",
            how="left",
        )

        merged_csv_gdf.drop(columns="GEOID_t", inplace=True)

        pass

    elif census_type == "b":

        csv = csv.add_suffix("_b")

        merged_csv_gdf = gdf.merge(
            csv,
            left_on="geo_id_b2020",
            right_on="GEOID_b",
            how="left",
        )

        merged_csv_gdf.drop(columns="GEOID_b", inplace=True)

        pass

    elif census_type == "bg":

        csv = csv.add_suffix("_bg")

        merged_csv_gdf = gdf.merge(
            csv,
            left_on="geo_id_bg2020",
            right_on="GEOID_bg",
            how="left",
        )

        merged_csv_gdf.drop(columns="GEOID_bg", inplace=True)

        pass

    else:
        raise ValueError

    return merged_csv_gdf

In [30]:
base_dataset = add_columns_from_census(base_dataset, acs_table_t, "t")
base_dataset = add_columns_from_census(base_dataset, acs_table_bg, "bg")

In [31]:
base_dataset.head()

,OBJECTID_1,OBJECTID,FEATURE_KE,PARCEL_TYP,REID,PARCEL_PK,AUT_SNAPSH,PIN,PIN_EXT,PIN_MAP,PIN_BLOCK,PIN_LOT,PROPERTY_D,LOCATION_A,OLD_MAP,VCS,CITY,CITY_PCNT,FIRE_DISTR,FIRE_PCNT,SPEC_DIST,SPEC_DIST_,TOWNSHIP,LAND_CLASS,ACREAGE,CALCULATED,DEEDED_ACR,PERMIT_DAT,PERMIT_NUM,ZONING,HISTORY_RE,HISTORY__1,ETJ,PROPERTY_O,OWNER_MAIL,OWNER_MA_1,OWNER_MA_2,OWNER_MA_3,OWNER_MA_4,OWNER_MA_5,OWNER_MA_6,PHYADDR_LO,PHYADDR_ST,PHYADDR_DI,PHYADDR__1,PHYADDR__2,PHYADDR__3,PHYADDR_CI,PHYADDR__4,PHYADDR_ZI,PHYADDR_PL,TOTAL_LAND,TOTAL_BLDG,LAND_USE_V,USE_VALUE_,HISTORIC_V,TOTAL_DEFE,TOTAL_PROP,COST_TOTAL,INCOME_TOT,SALES_COMP,VALUE_APPR,DEED_DATE,DEED_BOOK,DEED_PAGE,REVENUE_ST,PKG_SALE_D,PKG_SALE_P,LAND_SALE_,LAND_SAL_1,TOTAL_UNIT,RECYCLE_UN,APT_SC_SQR,HEATED_ARE,ELDERLY_EX,OTHER_EXMP,NEIGHBORHO,VETRANS_EX,DEED_PATH,ACCOUNT_TY,CITY_CODE,FIRE_DIS_1,PHYADDR__5,TOTAL_OBLD,EXEMPTION_,MAP_BOOK,MAP_PAGE,MAP_SCALE,MAP_ROD_DO,IS_PENDING,GROSS_LEAS,PLAT_BOOK,PLAT_PAGE,PHOTO_PATH,PHOTO_DATE,Shape__Are,Shape__Len,geometry,designation,housing_type,du_est_final,students2324,students2223,students2122,students2021,geo_id_b2010,geo_id_b2020,geo_id_bg2010,geo_id_bg2020,sch_id_base1819_es,sch_id_base_es,sch_id_gt_es,sch_id_yr_es,sch_id_yr_optout_es,sch_id_zone,sch_id_base_hs,sch_id_gt_hs,sch_id_base1819_ms,sch_id_base_ms,sch_id_gt_ms,sch_id_yr_ms,pu_2122_833,pu_2324_848,geo_id_t2010,geo_id_t2020,TOTAL_PROP_VALUE,estimate_rent_total_t,moe_rent_total_t,estimate_median_house_value_t,estimate_median_year_structure_build_t,estimate_housing_units_t,pct_vacant_t,pct_owner_occupied_t,estimate_rent_total_bg,moe_rent_total_bg,estimate_median_house_value_bg,estimate_median_year_structure_build_bg,estimate_housing_units_bg,pct_vacant_bg,pct_owner_occupied_bg
0,5293332.0,78584.0,78587.0,0.0,100000,1.0,2021-12-20,0822419440,000,0822,41,9440,PROP-DUKE UNIV/EAST CAMPU S-N/S W MAIN ST DUKE...,1500 W MAIN ST,None,C822N,DURHAM,100,None,None,CAPITAL FINANCE,100,NONE,CMNTY SVC/ COLLEGE,97.24,97.24,97.24,None,None,UC(D),None,None,DURHAM CITY,DUKE UNIVERSITY,"2200 WEST MAIN ST, STE 300",None,None,DURHAM,NC,27705,None,3339.0,1500,W,MAIN,ST,None,DURHAM,NC,27705,None,29471700.0,0.0,29471700.0,NaN,0.0,0.0,29471700.0,29471700.0,None,None,VLAPCOST,1997-01-01,000013,00335,0.0,None,NaN,None,NaN,None,None,None,NaN,None,29471700.0,W PETTIGREW ST,None,000013/,None,None,None,None,0.0,0200,None,None,00,None,N,0.0,000000,000000,None,None,4.241395e+06,8814.872258,"MULTIPOLYGON (((-78.91250 36.00508, -78.91253 ...",non-res,non-res,0,4,6,2,1,3.706300e+14,370630015031000,3.706300e+11,370630015031,363.0,363.0,363.0,364.0,NaN,347.0,365.0,365.0,306.0,306.0,306.0,370.0,16.0,15.0,3.706300e+10,37063001503,29471700.0,0.0,13.0,NaN,NaN,4.0,1.00,NaN,0.0,13.0,NaN,NaN,4.0,1.0,NaN
1,5293333.0,18719.0,18719.0,0.0,100001,2.0,2021-12-20,0822537639,000,0822,53,7639,THE SUBURBAN L&P CO/BLK:1 5/LT#01 PL00005A-000041,922 LANCASTER ST,None,R822D,DURHAM,100,None,None,CAPITAL FINANCE,100,NONE,RES/ 2-FAMILY,0.16,0.16,0.16,None,None,RU-5(2),None,None,DURHAM CITY,"WILKERSON, GLORIA MCBROOM",2429 BIVINS RD,None,None,DURHAM,NC,27712,None,3069.0,922,None,LANCASTER,ST,None,DURHAM,NC,27701,None,115900.0,45120.0,115900.0,NaN,0.0,0.0,161020.0,161020.0,None,None,VLAPCOST,2008-07-09,006008,00920,0.0,None,NaN,None,NaN,None,None,None,1216.0,None,0.0,WALL TOWN,None,006008/,None,None,None,None,0.0,None,None,None,00,None,N,0.0,00005A,000041,None,None,6.934546e+03,376.739897,"POLYGON ((-78.91321 36.01309, -78.91320 36.013...",multi,du_tri,2,0,0,0,0,3.706300e+14,370630003013002,3.706300e+11,370630003013,363.0,363.0,363.0,364.0,NaN,347.0,365.0,365.0,306.0,306.0,306.0,370.0,16.0,815.0,3.706300e+10,37063000301,161020.0,1168.0,330.0,225200.0,1966.0,1625.0,0.04,0.251282,372.0,158.0,425000.0,1942.0,453.0,0.0,0.178808
2,5293334.0,18271.0,18271.0,0.0,100002,3.0,2021-12-20,0822537634,000,0822,53,7634,THE SUBURBAN L&P CO/BLK:1 5/LT#03 PL00005A-000041,920 LANCASTER ST,None,R822D,DURHAM,100,None,None,CAPITAL FINANCE,100,NONE,RES/ 2-FAMILY,0.16,0.16,0.16,None,N

In [32]:
subset_list = [
    # durham open
    "OBJECTID_1",
    "OBJECTID",
    "REID",
    "PIN",
    "PROPERTY_D",
    "LOCATION_A",
    "SPEC_DIST",
    "LAND_CLASS",
    "ACREAGE",
    "PROPERTY_O",
    "OWNER_MAIL",
    "geometry",
    # du est
    "designation",
    "housing_type",
    "du_est_final",
    "students2324",
    "students2223",
    "students2122",
    "students2021",
    "geo_id_b2010",
    "geo_id_b2020",
    "geo_id_bg2010",
    "geo_id_bg2020",
    "sch_id_base1819_es",
    "sch_id_base_es",
    "sch_id_gt_es",
    "sch_id_yr_es",
    "sch_id_yr_optout_es",
    "sch_id_zone",
    "sch_id_base_hs",
    "sch_id_gt_hs",
    "sch_id_base1819_ms",
    "sch_id_base_ms",
    "sch_id_gt_ms",
    "sch_id_yr_ms",
    "pu_2122_833",
    "pu_2324_848",
    "geo_id_t2010",
    "geo_id_t2020",
    "TOTAL_PROP_VALUE",
    # census t
    "estimate_rent_total_t",
    "moe_rent_total_t",
    "estimate_median_house_value_t",
    "estimate_median_year_structure_build_t",
    "estimate_housing_units_t",
    "pct_vacant_t",
    "pct_owner_occupied_t",
    # bg
    "estimate_rent_total_bg",
    "moe_rent_total_bg",
    "estimate_median_house_value_bg",
    "estimate_median_year_structure_build_bg",
    "estimate_housing_units_bg",
    "pct_vacant_bg",
    "pct_owner_occupied_bg",
]
final_dataset = base_dataset[subset_list]

In [33]:
final_dataset

,OBJECTID_1,OBJECTID,REID,PIN,PROPERTY_D,LOCATION_A,SPEC_DIST,LAND_CLASS,ACREAGE,PROPERTY_O,OWNER_MAIL,geometry,designation,housing_type,du_est_final,students2324,students2223,students2122,students2021,geo_id_b2010,geo_id_b2020,geo_id_bg2010,geo_id_bg2020,sch_id_base1819_es,sch_id_base_es,sch_id_gt_es,sch_id_yr_es,sch_id_yr_optout_es,sch_id_zone,sch_id_base_hs,sch_id_gt_hs,sch_id_base1819_ms,sch_id_base_ms,sch_id_gt_ms,sch_id_yr_ms,pu_2122_833,pu_2324_848,geo_id_t2010,geo_id_t2020,TOTAL_PROP_VALUE,estimate_rent_total_t,moe_rent_total_t,estimate_median_house_value_t,estimate_median_year_structure_build_t,estimate_housing_units_t,pct_vacant_t,pct_owner_occupied_t,estimate_rent_total_bg,moe_rent_total_bg,estimate_median_house_value_bg,estimate_median_year_structure_build_bg,estimate_housing_units_bg,pct_vacant_bg,pct_owner_occupied_bg
0,5293332.0,78584.0,100000,0822419440,PROP-DUKE UNIV/EAST CAMPU S-N/S W MAIN ST DUKE...,1500 W MAIN ST,CAPITAL FINANCE,CMNTY SVC/ COLLEGE,97.24,DUKE UNIVERSITY,"2200 WEST MAIN ST, STE 300","MULTIPOLYGON (((-78.91250 36.00508, -78.91253 ...",non-res,non-res,0,4,6,2,1,3.706300e+14,370630015031000,3.706300e+11,370630015031,363.0,363.0,363.0,364.0,NaN,347.0,365.0,365.0,306.0,306.0,306.0,370.0,16.0,15.0,3.706300e+10,37063001503,29471700.0,0.0,13.0,NaN,NaN,4.0,1.000000,NaN,0.0,13.0,NaN,NaN,4.0,1.000000,NaN
1,5293333.0,18719.0,100001,0822537639,THE SUBURBAN L&P CO/BLK:1 5/LT#01 PL00005A-000041,922 LANCASTER ST,CAPITAL FINANCE,RES/ 2-FAMILY,0.16,"WILKERSON, GLORIA MCBROOM",2429 BIVINS RD,"POLYGON ((-78.91321 36.01309, -78.91320 36.013...",multi,du_tri,2,0,0,0,0,3.706300e+14,370630003013002,3.706300e+11,370630003013,363.0,363.0,363.0,364.0,NaN,347.0,365.0,365.0,306.0,306.0,306.0,370.0,16.0,815.0,3.706300e+10,37063000301,161020.0,1168.0,330.0,225200.0,1966.0,1625.0,0.040000,0.251282,372.0,158.0,425000.0,1942.0,453.0,0.000000,0.178808
2,5293334.0,18271.0,100002,0822537634,THE SUBURBAN L&P CO/BLK:1 5/LT#03 PL00005A-000041,920 LANCASTER ST,CAPITAL FINANCE,RES/ 2-FAMILY,0.16,SCANLON REALTY CORP,117 PARK AVENUE,"POLYGON ((-78.91274 36.01307, -78.91275 36.012...",multi,du_tri,2,0,0,0,0,3.706300e+14,370630003013002,3.706300e+11,370630003013,363.0,363.0,363.0,364.0,NaN,347.0,365.0,365.0,306.0,306.0,306.0,370.0,16.0,815.0,3.706300e+10,37063000301,180557.0,1168.0,330.0,225200.0,1966.0,1625.0,0.040000,0.251282,372.0,158.0,425000.0,1942.0,453.0,0.000000,0.178808
3,5293335.0,18958.0,100003,0822537539,PROP-PARRISH E J/W DURHAM /BLK:15/LT#05 PL0000...,918 LANCASTER ST,CAPITAL FINANCE,RES/ 2-FAMILY,0.16,RICHARD L SOLES TRUSTEE;JULIE H SOLES TRUSTEE;...,5607 RUSSELL RD,"POLYGON ((-78.91275 36.01293, -78.91276 36.012...",multi,du_tri,2,0,0,0,0,3.706300e+14,370630003013002,3.706300e+11,370630003013,363.0,363.0,363.0,364.0,NaN,347.0,365.0,365.0,306.0,306.0,306.0,370.0,16.0,815.0,3.706300e+10,37063000301,167187.0,1168.0,330.0,225200.0,1966.0,1625.0,0.040000,0.251282,372.0,158.0,425000.0,1942.0,453.0,0.000000,0.178808
4,5293336.0,17297.0,100004,0822537534,PROP-PARRISH E J/W DURHAM /BLK:15/LT#07 PL0000...,916 LANCASTER ST,CAPITAL FINANCE,RES/ 1-FAMILY,0.16,"LOUKAKIS, IOANNIS",916 LANCASTER ST,"POLYGON ((-78.91276 36.01280, -78.91276 36.012...",single,sf_detach,1,0,0,0,0,3.706300e+14,370630003013002,3.706300e+11,370630003013,363.0,363.0,363.0,364.0,NaN,347.0,365.0,365.0,306.0,306.0,306.0,370.0,16.0,815.0,3.706300e+10,37063000301,670055.0,1168.0,330.0,225200.0,1966.0,1625.0,0.040000,0.251282,372.0,158.0,425000.0,1942.0,453.0,0.000000,0.178808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121290,5419376.0,248171.0,236818,0832301704,PROP-MOHAMMAD RAFATI/LT#1B PL209-194,0 NO ADDRESS,CAPITAL FINANCE,RES/ 1-FAMILY,0.21,"RAFATI, MOHAMMAD",407 EDWARD ST,"POLYGON ((-78.88783 36.00471, -78.88783 36.004...",single,sf_detach,1,0,0,0,0,3.706300e+14,370630002002007,3.706300e+

In [34]:
final_dataset.to_csv("../data/final_base.csv", index=None)